# Lunita's Team:

* Cano Jeorval
* Cuevas Danilo
* Erosa Jorge
* Hernandez Andrés
* Robles Jack

# NB1. Network Statistics

Consider the following networks:

* **Facebook Northwester University**(socfb-Northwestern25.edges.gz). Network of Facebook users at Northwestern University. Nodes represent people, and links stand for Facebook friend connections.
* **US air transportation** (openflights_usa.edges.gz). The US air transportation network using flight data from OpenFlights.org. Nodes represent airports, and links stand for connections between them.
* **Twitter USA Politics**(retweet-digraph.edges.gz). Retweet directed network with weigtht on Twitter, among people sharing posts about US politics. Links represent retweets of posts that used different hashtags (#tcot, #p2). The direction of the link from user A to B indicates that a message has propagated from A to B.

In [1]:
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate

## Task 1
Create a table including the following characteristics for each network:
* Number of Nodes $N$.
* Number of Links $L$.
* Density $d$.
* Average Degree $\langle k\rangle $. 
* Clustering Coefficient $C_C$. 
    
Consider the following observations:
* In the case of undirected networks, compute the average in-degree.
* In the case of undirected networks, compute the clustering coefficient without taking into account the directions of the edges. In NetworkX it is possible to use the ``` G.to_undirected() ``` method to return an undirected copy of a graph G.


In [2]:
networks = ['Networks/socfb-Northwestern25.edgelist', 'Networks/openflights_usa.edges', 'Networks/retweet-digraph.edges']


G_fb = nx.read_edgelist(networks[0])
G_airports = nx.read_edgelist(networks[1])
G_twitter = nx.read_edgelist(networks[2], edgetype = float, data=(("weight", float),), create_using= nx.DiGraph())

In [3]:
network_names = ['Facebook Northwester University', 'US air transportation', 'Twitter USA Politics']

values_dict = dict()

for name, graph in zip(network_names, [G_fb, G_airports, G_twitter]):
    N = len(graph)
    
    L = graph.number_of_edges()
    
    d = nx.classes.function.density(graph) 
    
    if nx.is_directed(graph):
        #weight
        degrees = np.array(list(graph.degree(weight = 'weight')))
        k = np.mean(degrees[:,1].astype(float))
        
        copy = graph.to_undirected()
        Cc = nx.algorithms.cluster.average_clustering(copy, weight = 'weight')
        
    else:
        degrees = np.array(list(graph.degree))
        k = np.mean(degrees[:,1].astype(float))
        
        Cc = nx.algorithms.cluster.average_clustering(graph)
        
    values_dict[name] = {'Number of nodes': N, 'Number of links': L, 'density': d, 'Average degree': k, 'Cluster coefficient': Cc}

In [4]:
df = pd.DataFrame(values_dict).T

# displaying the DataFrame
print(tabulate(df, headers = 'keys', tablefmt = 'pipe'))

|                                 |   Number of nodes |   Number of links |     density |   Average degree |   Cluster coefficient |
|:--------------------------------|------------------:|------------------:|------------:|-----------------:|----------------------:|
| Facebook Northwester University |             10567 |            488337 | 0.00874757  |         92.4268  |           0.237991    |
| US air transportation           |               546 |              2781 | 0.0186914   |         10.1868  |           0.493045    |
| Twitter USA Politics            |             18470 |             48365 | 0.000141782 |          6.62231 |           0.000442431 |


The average shortest-path length is a common aggregate distance measure for Networks. It can be obtained by averaging the shortest-path lengths across all pairs of nodes. The definition of this distance-based measure assume that the shortest-path length is defined for each pair of nodes. If there is any pairs without a path, then the the average path length is not defined. One way to present this result is by measuring only on the giant component; for the directed network it is possible to consider directed paths in the giants strongly connected component. However, due to the number of possible pairs of nodes, the computing of the average shortest-path length can be computational extensive.

## Task 2
Create a function ``` average_path_length_sample(G, N_sample)``` to compute the average path length on a Network. The function must identify if the network is directed or not.  The following method can be useful: ``` G.is_directed()```. In the case of directed networks it should use the strongly connected component to compute it. On the other hand, if the network is undirected, it should use the giang connected component of the network. 

In order to compute the average path length on a sample. Make a sample of ```N_sample``` randomly chosen nodes on the connected component and compute the average path length using it.

The function must input ```G```a Network and ```N_sample```the number of nodes to be considered in the sample and output the average path length.

Compute the average path length of the three given networks and add them into the table using ```N_sample=1000```.

In [5]:
def average_path_length_sample(G, N_sample = 1000):
    """
    Function to copute the average path length of sample on a given graph 
    
    G: object graph
    
    N_sample: Number of nodes to select
    """
    if G.is_directed():
        strongly_connected_component = max(nx.strongly_connected_components(G), key=len)
        sample = np.random.choice(list(strongly_connected_component), size =  N_sample, replace = False)
          
        suma = 0
        i = 1
        if nx.classes.function.is_weighted(G):
            for initial, to_node in itertools.combinations(sample, 2):
                i += 1
                suma += nx.algorithms.shortest_paths.generic.shortest_path_length(G, initial, to_node, weight = 'weight')
                
            average_length = suma/(N_sample * (N_sample-1))



        else:
            
            for initial, to_node in itertools.combinations(sample, 2):
                suma += nx.algorithms.shortest_paths.generic.shortest_path_length(G, initial, to_node)
            
            average_length = suma/(N_sample * (N_sample-1))
            
        
    else:
        giant_connected_component = max(nx.connected_components(G), key=len)
        sample = np.random.choice(list(giant_connected_component), size =  N_sample, replace = False)
        
        suma = 0
        if nx.classes.function.is_weighted(G):
            for initial, to_node in itertools.combinations(sample, 2):
                suma += nx.algorithms.shortest_paths.generic.shortest_path_length(G, initial, to_node, weight = 'weight')
                
            average_lenht = 2 * suma/(N_sample * (N_sample-1))



        else:
            
            for initial, to_node in itertools.combinations(sample, 2):
                suma += nx.algorithms.shortest_paths.generic.shortest_path_length(G, initial, to_node)
            
            average_length = 2 * suma/(N_sample * (N_sample-1))
            
    return average_length

In [6]:
len(max(nx.connected_components(G_airports), key=len))

539

In [7]:
len(max(nx.connected_components(G_fb), key=len))

10537

In [8]:
len(max(nx.strongly_connected_components(G_twitter), key=len))

1457

### NOTE: since in the airports graph the maximum component is 539, we prefered to use 500 instead of 1000

In [9]:
avg_fb = average_path_length_sample(G_fb, 500)
avg_airports = average_path_length_sample(G_airports, 500)
avg_twitter = average_path_length_sample(G_twitter, 500)

In [10]:
values_dict_task2 = {
    'Facebook Northwester University': {'average path length': avg_fb},
    'US air transportation': {'average path length': avg_airports}, 
    'Twitter USA Politics': {'average path length': avg_twitter}
    
}

In [11]:
df_2 = pd.DataFrame(values_dict_task2).T

# displaying the DataFrame
print(tabulate(df_2, headers = 'keys', tablefmt = 'pipe'))

|                                 |   average path length |
|:--------------------------------|----------------------:|
| Facebook Northwester University |               2.73556 |
| US air transportation           |               3.19562 |
| Twitter USA Politics            |               2.96554 |


## Useful NetworkX Methods

* [Reading and writing graphs](https://networkx.github.io/documentation/networkx-1.9/reference/readwrite.html). Check the ```read_edgelist``` method.
* [Components](https://networkx.github.io/documentation/stable/reference/algorithms/component.html).

## References
[1] F. Mencszer, S. Fortunato, C. A. Davis (2020). A First Course in Network Science.